## Titanic overlevelse

Her har vi data om passasjerer på skipet Titanic og informasjon om hvorvidt de overlevde. I denne oppgaven skal du bruke passasjerdata for å teste om det er forskjeller i overlevelsen mellom passasjergrupper og predikere om passasjeren overlevde. 

In [82]:
# imports
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss

In [83]:
# les inn Titanic-data
titanic_df = pd.read_csv('data/titanic.csv')

Først endrer vi litt på data for å gjøre oppgaven enklere. Dette er ikke en god ide å gjøre når vi vil lage bestmulige modeller. 

In [84]:
# del data i mål-, og prediktorvariabler
X_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
y_col = 'Survived'
reduced_df = titanic_df.loc[:, X_cols + [y_col]].dropna()
X_df = pd.get_dummies(reduced_df.loc[:, X_cols], dtype='int')
X = X_df.values
y = reduced_df[y_col].values

Del data i trenings-, validerigns-, og testdata med størrelser 70%, 15%, 15% av data. 

In [85]:
# dele data i trenings, validerings og testdata
# generer X_train, X_val, X_test, y_train, y_val, y_test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

På treningsdata, test om menn og kvinner hadde forskjellige sannsynligheter for å overleve.

In [90]:
# test om menn og kvinner hadde forskjellig overlevelse
train_df = pd.DataFrame(X_train, columns = X_df.columns)
train_df['Survived'] = y_train

kjonn_overlevelse = pd.crosstab(train_df['Sex_female'], train_df['Survived'])
kjonn_test = stats.fisher_exact(kjonn_overlevelse)
kjonn_test

SignificanceResult(statistic=13.73054587688734, pvalue=9.399553467162986e-37)

På treningsdata, test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde. 

In [99]:
# test om de som overlevde hadde forskjellig mean alder enn de som ikke overlevde
alder_test = stats.ttest_ind(train_df.loc[train_df.loc[:, 'Survived'] == 0, 'Age'], 
                            train_df.loc[train_df.loc[:, 'Survived'] == 1, 'Age'])
alder_test

TtestResult(statistic=2.3814519197745625, pvalue=0.01762007031920793, df=497.0)

Tren en kNN-modell med k=1, k=10 og k=50 på treningsdata. Tren også en logistisk regresjon, naive Bayes modell, tilfeldig skog og supportvektormaskin på treningsdata.

In [100]:
# tren forskjellige modeller

models = {
    'kNN-1': KNeighborsClassifier(n_neighbors=1),
    'kNN-10': KNeighborsClassifier(n_neighbors=10),
    'kNN-50': KNeighborsClassifier(n_neighbors=50),
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(),
    'SVC': SVC(probability=True)
}

for model_name, model in models.items():
    model.fit(X_train, y_train)

Sorter de ulike modellene etter nøyaktighet på valideringsdata (`sklearn.metrics.accuracy_score`).

In [93]:
# sjekk nøyaktighet for valideringsdata
accuracies = {name: accuracy_score(y_val, model.predict(X_val)) for name, model in models.items()}
accuracies_df = pd.DataFrame(accuracies.items(), columns=['model', 'accuracy'])
accuracies_df.sort_values(by='accuracy', ascending=False, inplace=True)
print(accuracies_df)

                 model  accuracy
3  Logistic Regression  0.785047
5        Random Forest  0.775701
0                kNN-1  0.719626
2               kNN-50  0.663551
4          Naive Bayes  0.654206
6                  SVC  0.654206
1               kNN-10  0.644860


I stedet for nøyaktighet er det vanlig å bruke log-loss, som tar hensyn til en probabilistisk prediksjon. Sorter de ulike modellene etter log-loss (`sklearn.metrics.log_loss`). 

In [94]:
# sjekk log loss for valideringsdata
losses = {model: log_loss(y_val, model.predict_proba(X_val)) for name, model in models.items()}
losses_df = pd.DataFrame(losses, index=['loss']).transpose()
losses_df.sort_values(by='loss', inplace=True)
print(losses_df) 

                                                         loss
LogisticRegression(max_iter=200)                     0.502013
SVC(probability=True)                                0.630017
KNeighborsClassifier(n_neighbors=50)                 0.630977
(DecisionTreeClassifier(max_features='sqrt', ra...   1.475712
KNeighborsClassifier(n_neighbors=10)                 1.951143
MultinomialNB()                                      3.040359
KNeighborsClassifier(n_neighbors=1)                 10.105697


Velg ut den beste modellen (basert på log-loss) og sjekk hvor godt den generaliserer ved å regne ut nøyaktighet og log-loss på testdata. 

In [77]:
# sjekk generaliseringsevne
best_model = losses_df.index[0]
generalization_accuracy = accuracy_score(y_test, best_model.predict(X_test))
generalization_logloss = log_loss(y_test, best_model.predict_proba(X_test))
print('Nøyaktighet:', generalization_accuracy)
print('Log-loss:', generalization_logloss)

Nøyaktighet: 0.7592592592592593
Log-loss: 0.47191213662883486


Gi en oppsummering over hva du har gjort og hva resultatet var. 

Jeg trente ulike modeller, som kNN og logistisk regresjon. Vi sammenlignet dem basert på nøyaktighet og log-loss, og jeg sjekket deretter hvor godt den beste modellen presterte på testdataene.